# PDF App — Project Documentation & Repro Steps (CLEAN)

This notebook is cleaned for cloud sharing. It does **not** contain any secrets.

It reads two environment variables:
- `PDF_API_URL` — the public endpoint for your PDF fill service (e.g. https://api.example.com/fill)
- `PDF_API_TOKEN` — the bearer token for the service (set locally or in the runtime prior to running the test cell)

----


## Overview

- Purpose: provide a runnable demo & documentation of the PDF fill service.
- Use this notebook for demos, testing, and capturing run steps.



In [ ]:
# Auto-install helper packages into the kernel's Python environment (safe to run)
import sys, subprocess, pkgutil
for pkg in ('requests','ipykernel'):
    if pkgutil.find_loader(pkg) is None:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
    else:
        print(f"{pkg} already installed.")


## API Reference

This notebook expects the PDF service to accept a JSON body like:

```
{
  "templateId": "intake_v1",
  "fields": {"client_first_name":"Test","client_last_name":"User","DOB":"01/01/1990"}
}
```

The service should return JSON with either `pdfBase64` (base64-encoded PDF) or `sharepointUrl`.



In [ ]:
# Test client that reads endpoint and token from environment variables
import os, requests, base64, json
from IPython.display import IFrame, display

SERVICE_URL = os.environ.get('PDF_API_URL', '').strip()
TOKEN = os.environ.get('PDF_API_TOKEN', '').strip()

if not SERVICE_URL:
    raise RuntimeError("Please set the environment variable PDF_API_URL to your service endpoint. Example: export PDF_API_URL='https://api.example.com/fill'")

HEADERS = {"Content-Type": "application/json"}
if TOKEN:
    HEADERS["Authorization"] = f"Bearer {TOKEN}"
else:
    print('Warning: No PDF_API_TOKEN set. If your service requires auth, set PDF_API_TOKEN before running.')

payload = {
    "templateId": "intake_v1",
    "fields": {
        "client_first_name": "Test",
        "client_last_name": "User",
        "DOB": "01/01/1990"
    }
}

print('Posting to', SERVICE_URL)
r = requests.post(SERVICE_URL, json=payload, headers=HEADERS, timeout=30)
print('Status code:', r.status_code)

try:
    data = r.json()
except Exception:
    print('Non-JSON response:', r.text)
    raise

if data.get('pdfBase64'):
    b64 = data['pdfBase64']
    pdf_bytes = base64.b64decode(b64)
    out_path = 'test_output.pdf'
    with open(out_path, 'wb') as f:
        f.write(pdf_bytes)
    print('Saved PDF to', out_path)
    try:
        display(IFrame(out_path, width=700, height=400))
    except Exception as e:
        print('Could not display PDF inline:', e)
elif data.get('sharepointUrl'):
    print('PDF uploaded to SharePoint at:', data['sharepointUrl'])
else:
    print('Response JSON:', json.dumps(data, indent=2))


## Power Automate Notes

- Normalize dates before sending to the API.
- Avoid long `apply to each` loops; batch requests when possible.



## Troubleshooting

- If requests fails with SSL errors and you're using self-signed certs, either use a trusted cert or configure client accordingly.



## Next steps / TODO

- Add real template mapping files.
- Add CI smoke test to call the endpoint and verify a PDF is produced.
